In [1]:
import time

print("Last updated : {}".format(time.strftime("%Y.%m.%d %H:%M:%S")))

Last updated : 2018.02.22 21:29:25


# TensorFlow 기본 원리

+ (1) 그래프 준비하기 (construction)
+ (2) 그래프 실행하기 (execution)

---

# MNIST 기본 세팅

**필요한 모듈 임포트**

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

**tf 버전 확인**

In [3]:
tf.__version__

'1.3.0'

**mnist 데이터 다운로드**

In [4]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [5]:
print(len(mnist.train.images[0])) # 784
print(len(mnist.train.labels[0])) # 10

784
10


---

**01 placeholder**

In [6]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 784])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

# 3) Neural Networks + Xavier initialization

+ 처음부터 cost가 매우 낮게 시작한다.

## Part I. 그래프 그리기

**02 predictions**

In [7]:
# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

In [8]:
W1 = tf.get_variable("W1", shape=[784,256], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal(shape=[256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))

predictions = tf.matmul(L2, W3) + b3

---

**03 cost function and optimizer**

In [9]:
cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

**04 accuracy**

In [10]:
is_correct = tf.equal(tf.argmax(predictions , axis=1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

## Part II. 그래프 실행

**05 training epoch/batch**

In [11]:
epochs = 15
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))

Epoch: 0001 cost = 0.421630698
Epoch: 0002 cost = 0.298625762
Epoch: 0003 cost = 0.277421966
Epoch: 0004 cost = 0.277740605
Epoch: 0005 cost = 0.267546119
Epoch: 0006 cost = 0.255783739
Epoch: 0007 cost = 0.258395557
Epoch: 0008 cost = 0.242385979
Epoch: 0009 cost = 0.253042733
Epoch: 0010 cost = 0.245207643
Epoch: 0011 cost = 0.233354252
Epoch: 0012 cost = 0.232882243
Epoch: 0013 cost = 0.241551608
Epoch: 0014 cost = 0.236734395
Epoch: 0015 cost = 0.234820484


**06 test accuracy**

In [12]:
feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}
print("Accuracy:", sess.run(accuracy, feed_dict=feed_dict))

Accuracy: 0.9627


---